In [ ]:
!pip install torch transformers

In [ ]:
!pip install datasets  rouge-score nltk

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7b59cacc2934439e9fb9c0efbdc09326680876a2fec7378c6ab9e644d4a76334
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aswathm21adr005","key":"0734bfa12911610bec69d2219aae84e0"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d pariza/bbc-news-summary

Dataset URL: https://www.kaggle.com/datasets/pariza/bbc-news-summary
License(s): CC0-1.0
 79% 7.00M/8.91M [00:00<00:00, 70.0MB/s]
100% 8.91M/8.91M [00:00<00:00, 80.1MB/s]


In [ ]:
!unzip /content/bbc-news-summary.zip

Streaming output truncated to the last 5000 lines.
  inflating: BBC News Summary/Summaries/sport/363.txt  
  inflating: BBC News Summary/Summaries/sport/364.txt  
  inflating: BBC News Summary/Summaries/sport/365.txt  
  inflating: BBC News Summary/Summaries/sport/366.txt  
  inflating: BBC News Summary/Summaries/sport/367.txt  
  inflating: BBC News Summary/Summaries/sport/368.txt  
  inflating: BBC News Summary/Summaries/sport/369.txt  
  inflating: BBC News Summary/Summaries/sport/370.txt  
  inflating: BBC News Summary/Summaries/sport/371.txt  
  inflating: BBC News Summary/Summaries/sport/372.txt  
  inflating: BBC News Summary/Summaries/sport/373.txt  
  inflating: BBC News Summary/Summaries/sport/374.txt  
  inflating: BBC News Summary/Summaries/sport/375.txt  
  inflating: BBC News Summary/Summaries/sport/376.txt  
  inflating: BBC News Summary/Summaries/sport/377.txt  
  inflating: BBC News Summary/Summaries/sport/378.txt  
  inflating: BBC News Summary/Summaries/sport/379.txt

In [ ]:
import os
import re
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")


In [ ]:
def read_files_from_folder(folder_path):
    """
    Reads text files from subfolders in the specified folder path.

    Args:
        folder_path (str): The path to the main folder containing subfolders for different categories.

    Returns:
        list: A list of tuples, each containing the category, filename, and content of a file.
    """

    data = []
    categories = ['business', 'entertainment', 'politics', 'sport', 'tech']

    for category in categories:
        category_path = os.path.join(folder_path, category)
        if os.path.exists(category_path):
            print(f"Reading files from category: {category}")
            for filename in os.listdir(category_path):
                file_path = os.path.join(category_path, filename)
                if os.path.isfile(file_path):
                    try:
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()
                    except UnicodeDecodeError:
                        with open(file_path, 'r', encoding='ISO-8859-1') as file:
                            content = file.read()
                    data.append((category, filename, content))
        else:
            print(f"Category folder not found: {category_path}")

    return data

# Paths to the folders
articles_folder = '/content/bbc news summary/BBC News Summary/News Articles'
summaries_folder = '/content/bbc news summary/BBC News Summary/Summaries'

# Read the data
articles_data = read_files_from_folder(articles_folder)
summaries_data = read_files_from_folder(summaries_folder)

# Check if data is read correctly
print(f"Number of articles read: {len(articles_data)}")
print(f"Number of summaries read: {len(summaries_data)}")

# Create DataFrames
df_articles = pd.DataFrame(articles_data, columns=['Category', 'Filename', 'Article'])
df_summaries = pd.DataFrame(summaries_data, columns=['Category', 'Filename', 'Summary'])

# Merge the DataFrames on 'Category' and 'Filename'
data = pd.merge(df_articles, df_summaries, on=['Category', 'Filename'])
data.head()

Reading files from category: business
Reading files from category: entertainment
Reading files from category: politics
Reading files from category: sport
Reading files from category: tech
Reading files from category: business
Reading files from category: entertainment
Reading files from category: politics
Reading files from category: sport
Reading files from category: tech
Number of articles read: 2225
Number of summaries read: 2225


,Category,Filename,Article,Summary
0,business,364.txt,Durex maker SSL awaits firm bid\n\nUK condom m...,UK condom maker SSL International has refused ...
1,business,405.txt,House prices rebound says Halifax\n\nUK house ...,"UK house prices increased by 1.1% in December,..."
2,business,165.txt,Consumer spending lifts US growth\n\nUS econom...,US economic growth accelerated in the third qu...
3,business,448.txt,US Ahold suppliers face charges\n\nUS prosecut...,US prosecutors have charged nine food supplier...
4,business,131.txt,Russian oil merger excludes Yukos\n\nThe merge...,"""Obtaining majority control over Gazprom is th..."


In [ ]:
df = data[['Article', 'Summary']].copy() # Create a copy of the relevant columns (Article and Summary)
print(df.shape)
df.drop_duplicates(inplace=True)
df.dropna(axis=0,inplace=True)
print(df.shape)

(2225, 2)
(2127, 2)


In [ ]:
# Load the spaCy English model and set up the stop_words
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words

In [ ]:
# Setting up the contraction_mapping to get rid of contractions in the text
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
def text_cleaner(text, num):
    """
    Cleans a text by performing various preprocessing steps such as removing unnecessary elements and normalizing the text.

    Args:
        text (str): The input text to be cleaned.
        num (int): A flag indicating whether to remove stopwords (0) or not (1).

    Returns:
        str: The cleaned and processed text.
    """
    # Replace newline characters with spaces
    text = text.replace('\n', ' ').replace('\n\n', ' ')

    # Convert to lowercase
    newString = text.lower()

    # Remove content within parentheses
    newString = re.sub(r'\([^)]*\)', '', newString)

    # Remove double quotes
    newString = re.sub('"', '', newString)

    # Replace contractions with their full forms
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])

    # Remove "'s" from words
    newString = re.sub(r"'s\b", "", newString)

    # Remove non-alphabetic characters
    newString = re.sub("[^a-zA-Z]", " ", newString)

    # Replace multiple occurrences of 'm' with 'mm'
    newString = re.sub('[m]{2,}', 'mm', newString)

    # Tokenize and remove stop words (if num == 0)
    if num == 0:
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens = newString.split()

    # Keep only words with more than one character
    long_words = [i for i in tokens if len(i) > 1]
    return (" ".join(long_words)).strip()

In [ ]:
cleaned_texts = []

# clean articles
for t in df["Article"]:
    cleaned_texts.append(text_cleaner(t, 0)) # 0 to remove the stop words and get rid of unnecessary words

# clean summaries
cleaned_summaries = []

for s in df["Summary"]:
    cleaned_summaries.append(text_cleaner(s, 1)) #1 to retain most of the summarized text

# create new dataframe for the cleaned texts and summaries
dataframe = pd.DataFrame(columns=["text", "summary"])

# put cleaned version of texts and summaries into the dataframe
dataframe["text"] = cleaned_texts
dataframe["summary"] = cleaned_summaries

In [ ]:
dataframe.head()

,text,summary
0,durex maker ssl awaits firm bid uk condom make...,uk condom maker ssl international has refused ...
1,house prices rebound says halifax uk house pri...,uk house prices increased by in december the f...
2,consumer spending lifts growth economic growth...,us economic growth accelerated in the third qu...
3,ahold suppliers face charges prosecutors charg...,us prosecutors have charged nine food supplier...
4,russian oil merger excludes yukos merger russi...,obtaining majority control over gazprom is the...


In [ ]:
dataframe.replace('', np.nan, inplace=True)
dataframe.dropna(axis=0,inplace=True)

In [ ]:
def text_length_percentile(df, text_column, percentile):
    """
    Calculate the text length at which a given percentile of texts are below that length.

    Args:
    df (pandas.DataFrame): The DataFrame containing the text data.
    text_column (str): The column name containing the text data.
    percentile (float): The percentile to calculate (between 0 and 100).

    Returns:
    int: The text length at the given percentile.
    """
    # Calculate the length of each text
    text_lengths = df[text_column].apply(lambda x: len(x.split()))

    # Calculate the length at the given percentile
    length_at_percentile = np.percentile(text_lengths, percentile)

    return int(length_at_percentile)

percentile = 90

length_at_the_percentile = text_length_percentile(dataframe, 'text', percentile)
print(f"{percentile}% of the texts are below this length: {length_at_the_percentile}")

length_at_the_percentile = text_length_percentile(dataframe, 'summary', percentile)
print(f"{percentile}% of the summaries are below this length: {length_at_the_percentile}")

90% of the texts are below this length: 313
90% of the summaries are below this length: 269


In [ ]:
max_input_length = 335
max_target_length = 290

In [ ]:
cleaned_text = np.array(dataframe["text"])
cleaned_summary = np.array(dataframe["summary"])

# define lists for short text and short summaries
short_text = []
short_summary = []

# clean outliers
for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split()) <= max_target_length and len(cleaned_text[i].split()) <= max_input_length):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])

# change the dataframe
final_dataframe = pd.DataFrame({"source_text":short_text, "target_text":short_summary})

In [ ]:
dataframe.shape, final_dataframe.shape

((2127, 2), (1947, 2))

In [ ]:
%%time
def lemmatize_text(text):
    doc = nlp(text)
    # Return lemmatized text
    return ' '.join([token.lemma_ for token in doc])

# Apply lemmatization function to DataFrame column
final_dataframe['source_text'] = final_dataframe['source_text'].apply(lemmatize_text)
final_dataframe['target_text'] = final_dataframe['target_text'].apply(lemmatize_text)
final_dataframe.head()

CPU times: user 1min 37s, sys: 176 ms, total: 1min 37s
Wall time: 1min 47s


,source_text,target_text
0,durex maker ssl await firm bid uk condom maker...,uk condom maker ssl international have refuse ...
1,house price rebound say halifax uk house price...,uk house price increase by in december the fir...
2,consumer spending lift growth economic growth ...,we economic growth accelerate in the third qua...
3,ahold supplier face charge prosecutor charge f...,we prosecutor have charge nine food supplier w...
4,russian oil merger exclude yukos merger russia...,obtain majority control over gazprom be the be...


In [ ]:
final_dataframe["source_text"] = final_dataframe["source_text"].apply(lambda x: "summarize: " + x)

In [ ]:
final_dataframe.head()

,source_text,target_text
0,summarize: durex maker ssl await firm bid uk c...,uk condom maker ssl international have refuse ...
1,summarize: house price rebound say halifax uk ...,uk house price increase by in december the fir...
2,summarize: consumer spending lift growth econo...,we economic growth accelerate in the third qua...
3,summarize: ahold supplier face charge prosecut...,we prosecutor have charge nine food supplier w...
4,summarize: russian oil merger exclude yukos me...,obtain majority control over gazprom be the be...


In [ ]:
train_df, test_df = train_test_split(final_dataframe, test_size = 0.25)
train_df.shape, test_df.shape

((1460, 2), (487, 2))

In [ ]:
train_df.head()

,source_text,target_text
690,summarize: charity single quake relief singer ...,he say the song be slow ballad and would work ...
584,summarize: elvis regain chart position elvis p...,hit single one night will follow next week wit...
678,summarize: john peel replacement begin permane...,the permanent replacement for late dj john pee...
365,summarize: split cap pay compensation investor...,eighteen investment firm involve in the sale o...
799,summarize: super size win writer award super s...,spurlock be give his award on the same day the...


In [ ]:
import torch
from datasets import Dataset


In [ ]:
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# Load pre-trained Pegasus model and tokenizer for summarization
model_checkpoint = 'google/pegasus-xsum'  # You can use other pegasus models like 'google/pegasus-large' or 'google/pegasus-cnn_dailymail'
tokenizer = PegasusTokenizer.from_pretrained(model_checkpoint)
model = PegasusForConditionalGeneration.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["source_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True,padding='max_length')
    model_inputs["labels"] =   labels["input_ids"]
    return model_inputs

In [ ]:
train = Dataset.from_pandas(train_df)
valid = Dataset.from_pandas(test_df)

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/1460 [00:00<?, ? examples/s]

Map:   0%|          | 0/487 [00:00<?, ? examples/s]

In [ ]:
tokenized_valid

Dataset({
    features: ['source_text', 'target_text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 487
})

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Lower batch size
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Simulate larger batch size
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True  # Enable mixed precision
)

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid
)

trainer.train()




Epoch,Training Loss,Validation Loss
0,No log,6.525784


Epoch,Training Loss,Validation Loss
0,No log,6.525784
1,No log,6.269647
2,No log,6.199841


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6, 'forced_eos_token_id': 1}


TrainOutput(global_step=273, training_loss=7.094090276585394, metrics={'train_runtime': 1008.2348, 'train_samples_per_second': 4.344, 'train_steps_per_second': 0.271, 'total_flos': 4128999500021760.0, 'train_loss': 7.094090276585394, 'epoch': 2.9917808219178084})

In [ ]:
import gc
gc.collect()

6214

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!pip install evaluate
import evaluate

metric = evaluate.load("rouge")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
eval_dataset = Dataset.from_pandas(test_df)

In [ ]:
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)

Map:   0%|          | 0/487 [00:00<?, ? examples/s]